### 서울시 부동산 XML API 예제

In [1]:
# 라이브러리 가져오기
import lxml
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
import math

In [1]:
service_key = ''
url = f'http://openapi.seoul.go.kr:8088/{service_key}/xml/tbLnOpendataRtmsV/1/5/'
print(url)

http://openapi.seoul.go.kr:8088//xml/tbLnOpendataRtmsV/1/5/


In [11]:
req = requests.get(url)
req.content

b'<?xml version="1.0" encoding="UTF-8"?>\n<tbLnOpendataRtmsV>\n<list_total_count>2659001</list_total_count>\n<RESULT>\n<CODE>INFO-000</CODE>\n<MESSAGE>\xec\xa0\x95\xec\x83\x81 \xec\xb2\x98\xeb\xa6\xac\xeb\x90\x98\xec\x97\x88\xec\x8a\xb5\xeb\x8b\x88\xeb\x8b\xa4</MESSAGE>\n</RESULT>\n<row>\n<ACC_YEAR>2023</ACC_YEAR>\n<SGG_CD>11500</SGG_CD>\n<SGG_NM>\xea\xb0\x95\xec\x84\x9c\xea\xb5\xac</SGG_NM>\n<BJDONG_CD>10200</BJDONG_CD>\n<BJDONG_NM>\xeb\x93\xb1\xec\xb4\x8c\xeb\x8f\x99</BJDONG_NM>\n<LAND_GBN>1</LAND_GBN>\n<LAND_GBN_NM>\xeb\x8c\x80\xec\xa7\x80</LAND_GBN_NM>\n<BONBEON>0078</BONBEON>\n<BUBEON>0007</BUBEON>\n<BLDG_NM>\xea\xb8\xb8\xed\x9b\x88\xec\x97\x94\xed\x8a\xb8\xeb\x9f\xb0\xec\x8a\xa4\xeb\xb9\x8c</BLDG_NM>\n<DEAL_YMD>20230515</DEAL_YMD>\n<OBJ_AMT>16450</OBJ_AMT>\n<BLDG_AREA>27.29</BLDG_AREA>\n<TOT_AREA>40.420000</TOT_AREA>\n<FLOOR>11</FLOOR>\n<RIGHT_GBN/>\n<CNTL_YMD/>\n<BUILD_YEAR>2004</BUILD_YEAR>\n<HOUSE_TYPE>\xec\x98\xa4\xed\x94\xbc\xec\x8a\xa4\xed\x85\x94</HOUSE_TYPE>\n<REQ_GBN>\xe

In [12]:
soup = BeautifulSoup(req.content, "lxml") # XML 파싱
print(soup)

<?xml version="1.0" encoding="UTF-8"?><html><body><tblnopendatartmsv>
<list_total_count>2659001</list_total_count>
<result>
<code>INFO-000</code>
<message>정상 처리되었습니다</message>
</result>
<row>
<acc_year>2023</acc_year>
<sgg_cd>11500</sgg_cd>
<sgg_nm>강서구</sgg_nm>
<bjdong_cd>10200</bjdong_cd>
<bjdong_nm>등촌동</bjdong_nm>
<land_gbn>1</land_gbn>
<land_gbn_nm>대지</land_gbn_nm>
<bonbeon>0078</bonbeon>
<bubeon>0007</bubeon>
<bldg_nm>길훈엔트런스빌</bldg_nm>
<deal_ymd>20230515</deal_ymd>
<obj_amt>16450</obj_amt>
<bldg_area>27.29</bldg_area>
<tot_area>40.420000</tot_area>
<floor>11</floor>
<right_gbn></right_gbn>
<cntl_ymd></cntl_ymd>
<build_year>2004</build_year>
<house_type>오피스텔</house_type>
<req_gbn>직거래</req_gbn>
<rdealer_lawdnm></rdealer_lawdnm>
</row>
<row>
<acc_year>2023</acc_year>
<sgg_cd>11350</sgg_cd>
<sgg_nm>노원구</sgg_nm>
<bjdong_cd>10200</bjdong_cd>
<bjdong_nm>월계동</bjdong_nm>
<land_gbn>1</land_gbn>
<land_gbn_nm>대지</land_gbn_nm>
<bonbeon>0556</bonbeon>
<bubeon>0000</bubeon>
<bldg_nm>주공2</bldg_nm>

/usr/local/lib/python3.10/dist-packages/bs4/builder/__init__.py:545: XMLParsedAsHTMLWarning: It looks like you're parsing an XML document using an HTML parser. If this really is an HTML document (maybe it's XHTML?), you can ignore or filter this warning. If it's XML, you should know that using an XML parser will be more reliable. To parse this document as XML, make sure you have the lxml package installed, and pass the keyword argument `features="xml"` into the BeautifulSoup constructor.
  warnings.warn(


### 데이터 가공
- XML 형태를 pandas dataframe으로 변환

In [13]:
years            = soup.find_all('acc_year')         # 접수년월
sgg_cds          = soup.find_all('sgg_cd')           # 자치구코드
sgg_nms          = soup.find_all('sgg_nm')           # 자치구명
bjdong_cds       = soup.find_all('bjdong_cd')        # 법정동코드
bjdong_nms       = soup.find_all('bjdong_nm')        # 법정동명
land_gbns        = soup.find_all('land_gbn')         # 지번구분
land_gbn_nms     = soup.find_all('land_gbn_nm')      # 지번구분명
land_gbn_nms     = soup.find_all('land_gbn_nm')      # 지번구분명
bonbeons         = soup.find_all('bonbeon')          # 본번
bubeons          = soup.find_all('bubeon')           # 부번
bldg_nms         = soup.find_all('bldg_nm')          # 건물명
deal_ymds        = soup.find_all('deal_ymd')         # 계약일
obj_amts         = soup.find_all('obj_amt')          # 물건금액(만원)
bldg_areas       = soup.find_all('bldg_area')        # 건물면적(㎡)
tot_areas        = soup.find_all('tot_area')         # 토지면적(㎡)
floors           = soup.find_all('floor')            # 층
right_gbns       = soup.find_all('right_gbn')        # 권리구분
cntl_ymds        = soup.find_all('cntl_ymd')         # 취소일
build_years      = soup.find_all('build_years')      # 건축년도
house_types      = soup.find_all('house_type')       # 건물용도
req_gbn          = soup.find_all('req_gbn')          # 신고구분
rdealer_lawdnms  = soup.find_all('rdealer_lawdnm')   # 신고한 개업공인중개사 시군구명

In [14]:
req_gbn

[<req_gbn>직거래</req_gbn>,
 <req_gbn>중개거래</req_gbn>,
 <req_gbn>중개거래</req_gbn>,
 <req_gbn>직거래</req_gbn>,
 <req_gbn>중개거래</req_gbn>]

In [15]:
# 반복문 활용
year_list           = []
sgg_cd_list         = []
bldg_nm_list        = []
obj_amt_list        = []
house_type_list     = []
rdealer_lawdnm_list = []

for year, sgg_cd, bldg_nm, obj_amt, house_type, rdealer_lawdnm in zip(years, sgg_cds, bldg_nms, obj_amts, house_types, rdealer_lawdnms):
  year_list.append(year.get_text())
  sgg_cd_list.append(sgg_cd.get_text())
  bldg_nm_list.append(bldg_nm.get_text())
  obj_amt_list.append(obj_amt.get_text())
  house_type_list.append(house_type.get_text())
  rdealer_lawdnm_list.append(rdealer_lawdnm.get_text())

df = pd.DataFrame({
    "acc_year": year_list, 
    "sgg_cd": sgg_cd_list, 
    "bldg_nm" : bldg_nm_list,
    "obj_amt": obj_amt_list,
    "house_type" : house_type_list,
    "rdealer_lawdnm": rdealer_lawdnm_list
})

df

,acc_year,sgg_cd,bldg_nm,obj_amt,house_type,rdealer_lawdnm
0,2023,11500,길훈엔트런스빌,16450,오피스텔,
1,2023,11350,주공2,32500,아파트,서울 노원구
2,2023,11440,\t신촌르·메이에르타운(Ⅱ)\t,13300,오피스텔,서울 마포구
3,2023,11440,마포공덕파크팰리스Ⅱ,35000,오피스텔,
4,2023,11260,지은아트빌(555-11),30500,연립다세대,서울 중랑구


## JSON 형태

In [2]:
service_key = ''
url = f'http://openapi.seoul.go.kr:8088/{service_key}/json/tbLnOpendataRtmsV/1/5/'
print(url)

http://openapi.seoul.go.kr:8088//json/tbLnOpendataRtmsV/1/5/


In [17]:
req = requests.get(url)
content = req.json() # HTTP 요청의 본문을 JSON 형식으로 파싱하여 해당 데이터를 변환
content

{'tbLnOpendataRtmsV': {'list_total_count': 2659001,
  'RESULT': {'CODE': 'INFO-000', 'MESSAGE': '정상 처리되었습니다'},
  'row': [{'ACC_YEAR': '2023',
    'SGG_CD': '11500',
    'SGG_NM': '강서구',
    'BJDONG_CD': '10200',
    'BJDONG_NM': '등촌동',
    'LAND_GBN': '1',
    'LAND_GBN_NM': '대지',
    'BONBEON': '0078',
    'BUBEON': '0007',
    'BLDG_NM': '길훈엔트런스빌',
    'DEAL_YMD': '20230515',
    'OBJ_AMT': '16450',
    'BLDG_AREA': 27.29,
    'TOT_AREA': 40.42,
    'FLOOR': 11.0,
    'RIGHT_GBN': '',
    'CNTL_YMD': '',
    'BUILD_YEAR': '2004',
    'HOUSE_TYPE': '오피스텔',
    'REQ_GBN': '직거래',
    'RDEALER_LAWDNM': ''},
   {'ACC_YEAR': '2023',
    'SGG_CD': '11350',
    'SGG_NM': '노원구',
    'BJDONG_CD': '10200',
    'BJDONG_NM': '월계동',
    'LAND_GBN': '1',
    'LAND_GBN_NM': '대지',
    'BONBEON': '0556',
    'BUBEON': '0000',
    'BLDG_NM': '주공2',
    'DEAL_YMD': '20230515',
    'OBJ_AMT': '32500',
    'BLDG_AREA': 38.64,
    'TOT_AREA': 0.0,
    'FLOOR': 12.0,
    'RIGHT_GBN': '',
    'CNTL_YMD': '',

In [18]:
# 코드 직접 짜기
# json을 데이터프레임으로 변환

In [35]:
content['tbLnOpendataRtmsV']

{'list_total_count': 2659001,
 'RESULT': {'CODE': 'INFO-000', 'MESSAGE': '정상 처리되었습니다'},
 'row': [{'ACC_YEAR': '2023',
   'SGG_CD': '11500',
   'SGG_NM': '강서구',
   'BJDONG_CD': '10200',
   'BJDONG_NM': '등촌동',
   'LAND_GBN': '1',
   'LAND_GBN_NM': '대지',
   'BONBEON': '0078',
   'BUBEON': '0007',
   'BLDG_NM': '길훈엔트런스빌',
   'DEAL_YMD': '20230515',
   'OBJ_AMT': '16450',
   'BLDG_AREA': 27.29,
   'TOT_AREA': 40.42,
   'FLOOR': 11.0,
   'RIGHT_GBN': '',
   'CNTL_YMD': '',
   'BUILD_YEAR': '2004',
   'HOUSE_TYPE': '오피스텔',
   'REQ_GBN': '직거래',
   'RDEALER_LAWDNM': ''},
  {'ACC_YEAR': '2023',
   'SGG_CD': '11350',
   'SGG_NM': '노원구',
   'BJDONG_CD': '10200',
   'BJDONG_NM': '월계동',
   'LAND_GBN': '1',
   'LAND_GBN_NM': '대지',
   'BONBEON': '0556',
   'BUBEON': '0000',
   'BLDG_NM': '주공2',
   'DEAL_YMD': '20230515',
   'OBJ_AMT': '32500',
   'BLDG_AREA': 38.64,
   'TOT_AREA': 0.0,
   'FLOOR': 12.0,
   'RIGHT_GBN': '',
   'CNTL_YMD': '',
   'BUILD_YEAR': '1992',
   'HOUSE_TYPE': '아파트',
   'REQ_GBN

In [41]:
df = pd.DataFrame(content['tbLnOpendataRtmsV']['row'])
df.shape

(5, 21)